<a href="https://colab.research.google.com/github/vamshap/PySpark-Challenges/blob/main/Deal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as _sum, when
from pyspark.sql.functions import *

# Initialize Spark session
spark = SparkSession.builder.appName("DealsQuotaAnalysis").getOrCreate()

# Create Deals DataFrame
deals_data = [
    (101, 400000),
    (101, 300000),
    (201, 500000),
    (301, 500000)
]
deals_columns = ["employee_id", "deal_size"]
deals_df = spark.createDataFrame(deals_data, deals_columns)

# Create Sales_Quota DataFrame
quota_data = [
    (101, 500000),
    (201, 400000),
    (301, 600000)
]
quota_columns = ["employee_id", "Quota"]
quota_df = spark.createDataFrame(quota_data, quota_columns)

deals_df.show()
quota_df.show()

employee_df = deals_df.groupBy("employee_id").agg(_sum("deal_size").alias("total_deal_size"))

empQuota_df = employee_df.alias("employee").join(quota_df.alias("quota"),col("employee.employee_id")==col("quota.employee_id"),how="inner")

empQuota_df.show()

empQuota_df.withColumn("YesorNo",when(col("total_deal_size")>col("Quota"),"Yes").otherwise("No")).show()



+-----------+---------+
|employee_id|deal_size|
+-----------+---------+
|        101|   400000|
|        101|   300000|
|        201|   500000|
|        301|   500000|
+-----------+---------+

+-----------+------+
|employee_id| Quota|
+-----------+------+
|        101|500000|
|        201|400000|
|        301|600000|
+-----------+------+

+-----------+---------------+-----------+------+
|employee_id|total_deal_size|employee_id| Quota|
+-----------+---------------+-----------+------+
|        101|         700000|        101|500000|
|        201|         500000|        201|400000|
|        301|         500000|        301|600000|
+-----------+---------------+-----------+------+

+-----------+---------------+-----------+------+-------+
|employee_id|total_deal_size|employee_id| Quota|YesorNo|
+-----------+---------------+-----------+------+-------+
|        101|         700000|        101|500000|    Yes|
|        201|         500000|        201|400000|    Yes|
|        301|         500000| 